#Sparse Subspace Clustering (Vidal IEEE Trans. on PAMI 2013)

**References:**


1.   Main Paper (Vidal IEEE Trans. on PAMI 2013): https://arxiv.org/pdf/1203.1005.pdf
2.   Supplementary Paper (Vidal CVPR 2009): http://cis.jhu.edu/~ehsan/Downloads/SSC-CVPR09-Ehsan.pdf
3.   Spectral Clustering: http://people.csail.mit.edu/dsontag/courses/ml14/notes/Luxburg07_tutorial_spectral_clustering.pdf
4.   Spectral Clustering Code: https://juanitorduz.github.io/spectral_clustering/











Our objective is given a set of points drawn from a `union of subspaces`, we need to find the number of subspaces, their dimensions, a basis for each subspace, and the segmentation of the data.

Let all the points be from $D$ dimensional space, here, for our use, we consider the value of $D$ to be $10$. Let us consider that we have $N=100$ $D$ dimensional data points.

In [130]:
import numpy as np
import cvxpy as cp
from cvxpy.atoms.elementwise.power import power

##### Set Parameters of the data space

In [131]:
N = 178 # Total no of points
D = 13  # Dimension of space
K = 3 # Number of clusters

In [132]:
from sklearn.datasets import load_wine
wine_data = load_wine()
wine_X = np.asarray(wine_data.data)
input_data = wine_X.T

In the $Y_i$ numpy array, we have $N_i$, $D$ dimensional data points i.e. the array $Y$ contains the collection of data points. We need to find out the value of $n$ which is the number of subspaces from which they have been sampled, $d_i$ which is the dimension of each subspace, $A_i$ which will be the basis for each subspace.

The matrix $Y$ contains the data points arranged sequentially forming a matrix of shape $(D,N)$. The $Y$ matrix is of the form $[y[0],y[1], \ldots, y[N-1]], $where each $y_i$ denotes the vector of data point.

In [133]:
def find_sparse_sol(Y,i,N):
  if i is 0:
    Ybari = Y[:,1:N]
  if i is N-1:
    Ybari = Y[:,0:N-1]
  if i!=0 and i!=N-1:
    Ybari = np.concatenate((Y[:,0:i],Y[:,i+1:N]),axis=1)
  yi = Y[:,i].reshape(D,1)
  # this ci will contain the solution of the l1 optimisation problem  min |ci|1   yi = Ybari*ci
  ci = cp.Variable(shape=(N-1,1))
  constraint = [cp.sum(ci)==1] #Ybari * ci == yi, 
  obj = cp.Minimize(power(cp.norm(yi-Ybari*ci,2),2) + 1.957*cp.norm(ci,1))
  prob = cp.Problem(obj, constraint)
  prob.solve()
  return ci.value

In [134]:
ci = find_sparse_sol(input_data,2,N)
count = 0
for i in range(N-1):
  if ci[i,:]!=0:
    count += 1
print(count)

177


The function `find_sparse_sol` finds the coefficients for the equation $y_i == Y_{\hat{i}} * c_i$. The method will return the $c_i$ coefficient vector for the removal of the $y_i$ vector from the data matrix. This $c_i$ vector will contain non zero coefficients for all the other points which are in the same subspace as $y_i$. So, among the $N-1$ column entries, $N_i - 1$ entries will be non zero. The remaining entries will be $0$.

In [135]:
C = np.concatenate((np.zeros((1,1)),find_sparse_sol(input_data,0,N)),axis=0)

for i in range(1,N):
  ci = find_sparse_sol(input_data,i,N)
  zero_element = np.zeros((1,1))
  cif = np.concatenate((ci[0:i,:],zero_element,ci[i:N,:]),axis=0)
  C = np.concatenate((C,cif),axis=1)
print(C.shape)

(178, 178)


We now include a zero vector of size $(1,1)$ at the $i$ th position of $c_i$ to form $\hat{c_i}$ which we represent as *cif* in the code.
Then we concatenate $[\hat{c_1}$, $\hat{c_2}$, ..., $\hat{c_N}]$ to form the matrix $C$.  

$C$ is the *Matrix of Coefficients* and is of the form $C = [\hat{c_1}, \hat{c_2}, ..., \hat{c_N}] \in \mathbb{R}^{NXN}$.

In [136]:
Cp = np.zeros((C.shape[0], C.shape[1]))
for i in range(N):
  for j in range(N):
    Cp[i,j] = abs(C[i,j]) + abs(C[j,i])

print(Cp.shape)

(178, 178)


In [137]:
cz = 0
for i in range(Cp.shape[0]):
    for j in range(Cp.shape[1]):
        if Cp[i,j] < 1e-5:
            cz += 1
print(cz)

28954


In the above code block, we make $C$ a symmetric matrix by the operation $C_{ij} = |C_{ij}| + |C_{ji}|$. It is still a valid representation of the similarity since if $y_i$ can write itself as a linear combination of all the points in the same subspace including $y_j$, then $y_j$ can also be represented as a linear combination of all the other points belonging to the same subspace including $c_i$.

In [138]:
D = np.zeros((N,N))
sum_list=[]
for i in range(N):
  csum = np.sum(Cp[:,i],axis=0)
  sum_list.append(csum)

D = np.diag(sum_list)
print(D)

[[1.77566829 0.         0.         ... 0.         0.         0.        ]
 [0.         3.0934134  0.         ... 0.         0.         0.        ]
 [0.         0.         1.26613727 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 4.42537469 0.         0.        ]
 [0.         0.         0.         ... 0.         1.67255326 0.        ]
 [0.         0.         0.         ... 0.         0.         2.04561665]]


$D$ is the `degree matrix`. In this case, $D \in \mathbb{R}^{NxN}$ is a diagonal matrix with $D_{ii} = \sum_{j}C_{ij}$. 

In [139]:
L = np.subtract(D, Cp)
print(L.shape)

(178, 178)


This $L$ is the Laplacian matrix, which can be defined as $L = D - C$. Next, we calculate the `eigenvalues` and `eigenvectors` of the Laplacian matrix, which we will use for Spectral clustering of the data points. $L$ is a *positive, semi-definite matrix* this means all the eigenvalues of the matrix will be greater than equal to $0$.

### Perform Spectral Clustering with Laplacian Matrix L

In [140]:
from scipy import linalg

eigenvals, eigenvcts = linalg.eig(L)

eigenvals = np.real(eigenvals)
eigenvcts = np.real(eigenvcts)

eig = eigenvals.reshape((N,1))

In [141]:
eig_list=[]
for x in eig:
  eig_list.append(x[0])

eig_list.sort()
print(eig_list)

[4.184282759358809e-15, 0.40708692451637374, 0.4689746635084622, 0.5779516306481536, 0.6222191645656293, 0.6393113351725086, 0.6492978617248959, 0.6843528018994844, 0.690545674101907, 0.709360730413077, 0.7478787787578655, 0.75322502866369, 0.7709757585593179, 0.7838003988320603, 0.7860244068934218, 0.8026982465261218, 0.8068087656108573, 0.8338458324658787, 0.8438760753705311, 0.8557140461623491, 0.8658152130508587, 0.8811502501396328, 0.8918770281065276, 0.9013866714266908, 0.9085650395681989, 0.9110336394949251, 0.9163963554568864, 0.9367961698611749, 0.940351622541126, 0.9448451430546264, 0.9547682541964578, 0.9598666369317627, 0.9719457090044502, 0.9754764391914685, 0.9846998246761173, 0.9923493010222075, 1.0102698299372492, 1.0163194252156498, 1.020447967076858, 1.0430171613351857, 1.0519454994953612, 1.0654818694643904, 1.0723215089860185, 1.0779668301340195, 1.0847314432839819, 1.097317237880487, 1.1235835005195878, 1.129085683154003, 1.1350664230869687, 1.1407700449048173, 1.1

Here, we sort all the eigenvalues of the $L$ matrix. We find that the number of zero eigenvalues is $2$, which are the first $2$ eigenvalues, which means that the graph has $2$ `connected components`, which is the **same as the number of subspaces** from which the data have been collected. 

> **Thus, the multiplicity of the zero eigenvalues of the Laplacian Matrix $L$ is equal to the *number of connected components in the graph* which is same as the number of subspaces.**

Sort Eigen Values

In [142]:
eigenvals_sorted_indices = np.argsort(eigenvals)
eigenvals_sorted = eigenvals[eigenvals_sorted_indices]

In [143]:
indices = []
for i in range(0,K):
    ind = []
    print(eigenvals_sorted_indices[i])
    ind.append(eigenvals_sorted_indices[i])
    indices.append(np.asarray(ind))

3
52
62


In [144]:
indices = np.asarray(indices)

In [145]:
zero_eigenvals_index = np.array(indices)

Here, the indices are arranged according to their sorted order of values and the sorted eigen values are stored in *eigenvals_sorted*.

In [146]:
eigenvals[zero_eigenvals_index]

array([[4.18428276e-15],
       [4.07086925e-01],
       [4.68974664e-01]])

Here, we collect the indices of *the zero eigen values* (for convenience we consider values smaller than $10^{-5}$ to be equal to zero). \\
*The zero eigen values* are the displayed.

In [147]:
import pandas as pd

proj_df = pd.DataFrame(eigenvcts[:, zero_eigenvals_index.squeeze()])
proj_df.columns = ['v_' + str(c) for c in proj_df.columns]
proj_df.head()

,v_0,v_1,v_2
0,0.074953,-0.070832,-0.096227
1,0.074953,-0.056307,-0.047522
2,0.074953,-0.058465,-0.019998
3,0.074953,-0.062030,-0.080295
4,0.074953,-0.012187,-0.034480


Stack the Eigen Vectors corresponding to the zero Eigen Values in a dataframe *proj_df*. This can be thought of as a $N X Z$ matrix ($Z$ = number of zero eigen values) where the columns denote an eigen vector and the rows denote a data point.

Apply *K-Means Clustering* with $K = 3$.

In [148]:
from sklearn.cluster import KMeans

def run_k_means(df, n_clusters):
    k_means = KMeans(random_state=25, n_clusters=n_clusters)
    k_means.fit(df)
    cluster = k_means.predict(df)
    return cluster

cluster = run_k_means(proj_df, n_clusters=K)

*run_k_means* applies `K-Means Clustering` on *proj_df* with number of clusters = $2$. \\
The clustering of the data points is returned in the variable *'cluster'*.

Display clusters formed

In [149]:
print(cluster)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 0 2 2 2 0 2 0 2 2 2 2 0
 0 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 0 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2
 2 0 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


As we can see, the data points have been clustered into two subspaces: $0$ and $1$ corresponding to the $2$ subspaces that we have considered.

In [150]:
c0 = 0
for l in cluster:
  if l == 0:
    c0 += 1
print(c0)

68


In [151]:
c1 = 0
for l in cluster:
  if l == 1:
    c1 += 1
print(c1)

49


In [152]:
c2 = 0
for l in cluster:
  if l == 2:
    c2 += 1
print(c2)

61


So we find that as expected $50$ data points have been labelled $0$, signifying that they belong to  the $1$st subspace, while the remaining $50$ points must belong to the $2$nd subspace. 

In [153]:
orig = wine_data.target

In [154]:
pred = np.asarray(cluster)

In [155]:
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import normalized_mutual_info_score
print("ARI = " + str(adjusted_rand_score(orig, pred)))
print("NMI = " + str(normalized_mutual_info_score(orig, pred)))

ARI = 0.8297415339237041
NMI = 0.8358558565678333
